# 🎯 InternVL vs Qwen-VL 模型对比教程

**欢迎来到视觉语言模型对比教程！**

本教程将指导您对比InternVL和Qwen-VL两个SOTA视觉语言模型的性能和特点。

---

## 📚 学习目标

- ✅ 理解InternVL和Qwen-VL的特点
- ✅ 对比两个模型在不同任务上的表现
- ✅ 学会根据场景选择合适的模型
- ✅ 掌握模型推理和评估方法

---

## ⏱️ 预计学习时间

- 完整运行：约 45-60 分钟
- 快速浏览：约 15-20 分钟

---

## 📌 前置要求

```bash
# 安装依赖
pip install torch transformers>=4.37.2 accelerate pillow matplotlib
pip install transformers_stream_generator  # For Qwen-VL
```

**硬件要求**：
- GPU: 16GB+ 显存（推荐）
- CPU: 可运行但较慢

---

# 第一部分：环境准备

## 1.1 导入必要的库

In [ ]:
# 基础库
import os
import sys
import time
import warnings
warnings.filterwarnings('ignore')

# 添加项目路径
sys.path.append('..')

# 深度学习
import torch
from transformers import (
    AutoModelForImageTextToText,  # For InternVL
    AutoModelForCausalLM,         # For Qwen-VL
    AutoProcessor,
    AutoTokenizer
)

# 图像处理
from PIL import Image
import matplotlib.pyplot as plt

# 进度条
from tqdm.auto import tqdm

print("✅ 所有库导入成功！")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 1.2 配置参数

In [ ]:
# 配置类
class Config:
    # 模型配置
    internvl_model = "OpenGVLab/InternVL2-8B"
    qwenvl_model = "Qwen/Qwen-VL-Chat"
    
    # 设备配置
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = "bfloat16" if torch.cuda.is_available() else "float32"
    
    # 推理配置
    max_new_tokens = 512
    temperature = 0.7
    
    # 测试图像目录
    image_dir = "./test_images"

config = Config()
print(f"📋 设备: {config.device}")
print(f"📋 精度: {config.dtype}")
print(f"📋 InternVL模型: {config.internvl_model}")
print(f"📋 Qwen-VL模型: {config.qwenvl_model}")

## 1.3 准备测试图像

我们提供三种方式准备测试图像:
1. **使用示例图像** - 从HuggingFace下载示例图像
2. **使用本地图像** - 使用您自己的图像文件
3. **创建测试图像** - 生成简单的测试图像(用于快速演示)

In [ ]:
import requests
from pathlib import Path
from io import BytesIO

def prepare_test_images():
    """准备测试图像"""
    # 创建图像目录
    os.makedirs(config.image_dir, exist_ok=True)
    
    # 方法1: 从网络下载示例图像 (推荐)
    sample_urls = [
        "https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/vision-image-colorization/data/tower.jpg",
        "https://raw.githubusercontent.com/pytorch/pytorch.github.io/master/assets/images/cat.jpg"
    ]
    
    test_images = []
    print("📥 下载测试图像...")
    
    for idx, url in enumerate(sample_urls):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            # 保存图像
            image_path = os.path.join(config.image_dir, f"test_{idx+1}.jpg")
            img = Image.open(BytesIO(response.content))
            img.save(image_path)
            test_images.append(image_path)
            print(f"✅ 下载成功: {image_path}")
        except Exception as e:
            print(f"⚠️ 下载失败 {url}: {e}")
    
    # 方法2: 如果下载失败,创建测试图像
    if not test_images:
        print("⚠️ 网络图像下载失败,创建本地测试图像...")
        for idx in range(2):
            image_path = os.path.join(config.image_dir, f"test_{idx+1}.jpg")
            # 创建彩色测试图像
            import numpy as np
            colors = [(255, 0, 0), (0, 255, 0)]  # 红色和绿色
            img_array = np.ones((224, 224, 3), dtype=np.uint8) * np.array(colors[idx])
            img = Image.fromarray(img_array)
            img.save(image_path)
            test_images.append(image_path)
            print(f"✅ 创建测试图像: {image_path}")
    
    # 方法3: 检查用户是否有本地图像
    local_images = list(Path(config.image_dir).glob("*.jpg")) + list(Path(config.image_dir).glob("*.png"))
    if local_images and not test_images:
        test_images = [str(p) for p in local_images[:2]]
        print(f"✅ 使用本地图像: {len(test_images)}张")
    
    return test_images

# 准备测试图像
test_images = prepare_test_images()
print(f"\n🎉 测试图像准备完成! 共{len(test_images)}张")

# 更新配置
config.test_images = test_images

# 显示测试图像
if test_images:
    fig, axes = plt.subplots(1, len(test_images), figsize=(12, 4))
    if len(test_images) == 1:
        axes = [axes]
    for idx, img_path in enumerate(test_images):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f"Test Image {idx+1}")
        axes[idx].axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 没有可用的测试图像,请手动准备图像并放到test_images目录")

---

# 第二部分：加载模型

## 2.1 加载InternVL模型

In [ ]:
print("🚀 加载InternVL模型...")

# 设置精度
if config.dtype == "bfloat16":
    dtype = torch.bfloat16
elif config.dtype == "float16":
    dtype = torch.float16
else:
    dtype = torch.float32

# 加载InternVL
internvl_processor = AutoProcessor.from_pretrained(
    config.internvl_model,
    trust_remote_code=True
)

internvl_model = AutoModelForImageTextToText.from_pretrained(
    config.internvl_model,
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
).eval()

print("✅ InternVL模型加载成功！")

## 2.2 加载Qwen-VL模型

In [ ]:
print("🚀 加载Qwen-VL模型...")

# 加载Qwen-VL
qwenvl_tokenizer = AutoTokenizer.from_pretrained(
    config.qwenvl_model,
    trust_remote_code=True
)

qwenvl_model = AutoModelForCausalLM.from_pretrained(
    config.qwenvl_model,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=dtype
).eval()

print("✅ Qwen-VL模型加载成功！")

---

# 第三部分：定义推理函数

## 3.1 InternVL推理函数

In [ ]:
def internvl_inference(image_path, question):
    """InternVL推理"""
    # 加载图像
    image = Image.open(image_path).convert('RGB')
    
    # 准备消息
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": question}
            ]
        }
    ]
    
    # 应用聊天模板
    prompt = internvl_processor.apply_chat_template(
        messages,
        add_generation_prompt=True
    )
    
    # 处理输入
    inputs = internvl_processor(
        text=prompt,
        images=[image],
        return_tensors="pt"
    )
    
    # 移动到设备
    inputs = {k: v.to(config.device) if isinstance(v, torch.Tensor) else v
             for k, v in inputs.items()}
    
    # 计时开始
    start_time = time.time()
    
    # 生成
    with torch.no_grad():
        outputs = internvl_model.generate(
            **inputs,
            max_new_tokens=config.max_new_tokens,
            temperature=config.temperature,
            do_sample=True
        )
    
    # 计时结束
    inference_time = time.time() - start_time
    
    # 解码
    response = internvl_processor.batch_decode(
        outputs,
        skip_special_tokens=True
    )[0]
    
    # 提取回答
    if "assistant\n" in response:
        answer = response.split("assistant\n")[-1].strip()
    else:
        answer = response.strip()
    
    return answer, inference_time

print("✅ InternVL推理函数定义完成")

## 3.2 Qwen-VL推理函数

In [ ]:
def qwenvl_inference(image_path, question):
    """Qwen-VL推理"""
    # 构建查询
    query = qwenvl_tokenizer.from_list_format([
        {'image': image_path},
        {'text': question},
    ])
    
    # 计时开始
    start_time = time.time()
    
    # 生成响应
    response, _ = qwenvl_model.chat(
        qwenvl_tokenizer,
        query=query,
        history=None,
        max_length=config.max_new_tokens
    )
    
    # 计时结束
    inference_time = time.time() - start_time
    
    return response, inference_time

print("✅ Qwen-VL推理函数定义完成")

---

# 第四部分：任务对比

## 4.1 图像描述生成对比

In [ ]:
# 选择测试图像
test_image = config.test_images[0]

# 问题
question = "Please describe this image in detail."

print("🖼️ 图像描述生成对比\n")
print(f"测试图像: {test_image}")
print(f"问题: {question}\n")

# InternVL
print("📝 InternVL:")
internvl_answer, internvl_time = internvl_inference(test_image, question)
print(f"回答: {internvl_answer}")
print(f"耗时: {internvl_time:.2f}秒\n")

# Qwen-VL
print("📝 Qwen-VL:")
qwenvl_answer, qwenvl_time = qwenvl_inference(test_image, question)
print(f"回答: {qwenvl_answer}")
print(f"耗时: {qwenvl_time:.2f}秒\n")

# 显示图像
img = Image.open(test_image)
plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')
plt.title('Test Image')
plt.show()

# 性能对比
print("⚡ 性能对比:")
print(f"InternVL耗时: {internvl_time:.2f}秒")
print(f"Qwen-VL耗时: {qwenvl_time:.2f}秒")
print(f"速度比: {internvl_time/qwenvl_time:.2f}x")

## 4.2 中文场景对比

In [ ]:
# 中文问题
chinese_question = "详细描述这张图片的内容"

print("🇨🇳 中文场景对比\n")
print(f"问题: {chinese_question}\n")

# InternVL
print("📝 InternVL:")
internvl_answer_cn, internvl_time_cn = internvl_inference(test_image, chinese_question)
print(f"回答: {internvl_answer_cn}")
print(f"耗时: {internvl_time_cn:.2f}秒\n")

# Qwen-VL
print("📝 Qwen-VL:")
qwenvl_answer_cn, qwenvl_time_cn = qwenvl_inference(test_image, chinese_question)
print(f"回答: {qwenvl_answer_cn}")
print(f"耗时: {qwenvl_time_cn:.2f}秒\n")

## 4.3 OCR识别对比

In [ ]:
# OCR问题
ocr_question = "Please extract all text from this image."

print("🔍 OCR识别对比\n")
print(f"问题: {ocr_question}\n")

# InternVL
print("📝 InternVL:")
internvl_ocr, internvl_time_ocr = internvl_inference(test_image, ocr_question)
print(f"识别结果: {internvl_ocr}")
print(f"耗时: {internvl_time_ocr:.2f}秒\n")

# Qwen-VL
print("📝 Qwen-VL:")
qwenvl_ocr, qwenvl_time_ocr = qwenvl_inference(test_image, ocr_question)
print(f"识别结果: {qwenvl_ocr}")
print(f"耗时: {qwenvl_time_ocr:.2f}秒\n")

---

# 第五部分：性能统计

## 5.1 推理时间对比

In [ ]:
import numpy as np

# 统计数据
tasks = ['Image Caption', 'Chinese', 'OCR']
internvl_times = [internvl_time, internvl_time_cn, internvl_time_ocr]
qwenvl_times = [qwenvl_time, qwenvl_time_cn, qwenvl_time_ocr]

# 绘图
x = np.arange(len(tasks))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, internvl_times, width, label='InternVL', color='#4CAF50')
bars2 = ax.bar(x + width/2, qwenvl_times, width, label='Qwen-VL', color='#2196F3')

ax.set_xlabel('Tasks', fontsize=12)
ax.set_ylabel('Inference Time (seconds)', fontsize=12)
ax.set_title('Inference Time Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(tasks)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# 添加数值标签
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}s',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# 打印统计
print("\n📊 平均推理时间:")
print(f"InternVL: {np.mean(internvl_times):.2f}秒")
print(f"Qwen-VL: {np.mean(qwenvl_times):.2f}秒")

---

# 🎓 总结

## ✅ 完成的内容

1. ✅ 加载InternVL和Qwen-VL模型
2. ✅ 对比图像描述生成能力
3. ✅ 对比中文理解能力
4. ✅ 对比OCR识别能力
5. ✅ 分析推理性能

## 🔑 关键发现

### InternVL优势
- ✅ 英文任务表现优秀
- ✅ 高分辨率图像支持
- ✅ OCR准确率更高
- ✅ 接近GPT-4V性能

### Qwen-VL优势
- ✅ 中文场景优化
- ✅ 推理速度较快
- ✅ 模型体积更小
- ✅ 部署更便捷

## 🎯 选型建议

- **英文为主**: 选择InternVL
- **中文为主**: 选择Qwen-VL
- **高性能要求**: 选择InternVL
- **资源受限**: 选择Qwen-VL

## 📚 参考资源

- [InternVL详解文档](../docs/01-模型调研与选型/08-InternVL模型详解.md)
- [Qwen-VL详解文档](../docs/01-模型调研与选型/07-Qwen-VL模型详解.md)
- [性能对比文档](../docs/01-模型调研与选型/09-InternVL与Qwen-VL对比.md)

---

**🎉 恭喜完成本教程！**

如有问题,欢迎在GitHub上提Issue。